In [1]:
import os
import pandas as pd

# feature extraction
from pliers.stimuli import VideoStim
from pliers.filters import FrameSamplingFilter
from pliers.extractors import FaceRecognitionFaceLocationsExtractor

from pliers.extractors import merge_results
from pliers import config

config.set_option("progress_bar", False)

In [2]:
deriv_dir = "/home/exp-psy/Desktop/study_face_tracks/derivatives"
out_dir = os.path.join(deriv_dir, "reference_no-faces")
os.makedirs(out_dir, exist_ok=True)

In [3]:
face_ext = FaceRecognitionFaceLocationsExtractor(model="cnn")
filt = FrameSamplingFilter(hertz=1)

In [4]:
for run in range(1, 9):
    video_f = os.path.join(deriv_dir, "fgav", f"fg_av_ger_seg{run}.mkv")
    video = VideoStim(video_f)
    
    print(f"starting with segment:\t {video_f}")
    
    # classify each second
    selected_frames = filt.transform(video)
    face_features = face_ext.transform(selected_frames)
    
    # adjust and save face df
    merged_faces = merge_results(face_features, metadata=False)
    merged_faces = merged_faces.drop(columns=["order"])
    merged_faces.to_csv(os.path.join(out_dir, f"run-0{run}_per-frame-faces.tsv"), sep="\t")
    
    # generate a "no-face" df
    gaps = []
    prev_end = 0.0
    for i, row in merged_faces.iterrows():
        current_onset = row["onset"]
        if current_onset > prev_end:
            gap = {                "onset": prev_end,
                "duration": current_onset - prev_end,
                "trial_type": f"no_face_{len(gaps)+1}"
            }
            gaps.append(gap)
        prev_end = max(prev_end, row["onset"] + row["duration"])
    
    # save the "no-fce" df
    marged_no_faces = pd.DataFrame(gaps)
    marged_no_faces.to_csv(os.path.join(out_dir, f"run-0{run}_no-faces.tsv"), sep="\t")

starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg1.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg2.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg3.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg4.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg5.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg6.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg7.mkv
starting with segment:	 /home/exp-psy/Desktop/study_face_tracks/derivatives/fgav/fg_av_ger_seg8.mkv
